# grafos

grafos são estruturas de dados compostas por **vértices** (ou nós) e **arestas** (ou edges) que conectam esses vértices.

diferente de árvores, grafos podem ter ciclos — ou seja, é possível começar em um vértice e voltar para ele mesmo seguindo as arestas.

grafos são usados para modelar relações entre objetos, como:
- redes sociais (pessoas e suas conexões)
- mapas e rotas (cidades e estradas)
- internet (páginas e links)
- dependências de pacotes (bibliotecas e suas dependências)

## terminologia básica

- **vértice (vertex/node):** um ponto no grafo
- **aresta (edge):** uma conexão entre dois vértices
- **grau (degree):** quantidade de arestas conectadas a um vértice
- **caminho (path):** sequência de vértices conectados por arestas
- **ciclo (cycle):** caminho que começa e termina no mesmo vértice
- **adjacência:** dois vértices são adjacentes se existe uma aresta entre eles

# tipos de grafos

## grafo não-direcionado (undirected graph)

as arestas não têm direção — se A está conectado a B, então B também está conectado a A.

```
    A --- B
    |     |
    C --- D
```

## grafo direcionado (directed graph / digraph)

as arestas têm direção — se existe uma aresta de A para B, não significa que existe uma de B para A.

```
    A --> B
    |     |
    v     v
    C --> D
```

## grafo ponderado (weighted graph)

cada aresta tem um peso ou custo associado.  
útil para modelar distâncias, custos, tempo de viagem, etc.

```
    A --5-- B
    |       |
    3       2
    |       |
    C --4-- D
```

## grafo cíclico vs acíclico

- **cíclico:** contém pelo menos um ciclo
- **acíclico:** não contém ciclos (DAG — Directed Acyclic Graph)

DAGs são muito usados em:
- sistemas de build (dependências)
- scheduling de tarefas
- blockchain

# representações de grafos

existem duas formas principais de representar grafos em memória: **matriz de adjacência** e **lista de adjacência**.

## matriz de adjacência

uma matriz 2D onde `matrix[i][j] = 1` se existe uma aresta entre o vértice `i` e o vértice `j`.

```
grafo:    A --- B
          |     |
          C --- D

vértices: A=0, B=1, C=2, D=3

matriz:
    A  B  C  D
A [ 0, 1, 1, 0 ]
B [ 1, 0, 0, 1 ]
C [ 1, 0, 0, 1 ]
D [ 0, 1, 1, 0 ]
```

**vantagens:**
- verificar se existe aresta entre dois vértices é `O(1)`
- simples de implementar

**desvantagens:**
- usa `O(V²)` de memória, onde V é o número de vértices
- ineficiente para grafos esparsos (poucos edges)

## lista de adjacência

um dicionário (ou array de listas) onde cada vértice aponta para uma lista de seus vizinhos.

```
grafo:    A --- B
          |     |
          C --- D

lista:
A: [B, C]
B: [A, D]
C: [A, D]
D: [B, C]
```

**vantagens:**
- usa `O(V + E)` de memória, onde E é o número de arestas
- eficiente para grafos esparsos
- fácil de iterar sobre os vizinhos

**desvantagens:**
- verificar se existe aresta entre dois vértices é `O(grau do vértice)`

## implementação: lista de adjacência

a forma mais comum de representar grafos em python é usando um dicionário.  
cada chave é um vértice e o valor é uma lista (ou set) de vizinhos.

In [1]:
from collections import defaultdict


class Grafo:
    def __init__(self, direcionado: bool = False):
        self.adj: dict[str, list[str]] = defaultdict(list)
        self.direcionado = direcionado

    def adicionar_aresta(self, u: str, v: str):
        self.adj[u].append(v)

        if not self.direcionado:
            self.adj[v].append(u)

    def vizinhos(self, v: str) -> list[str]:
        return self.adj[v]

    def vertices(self) -> list[str]:
        return list(self.adj.keys())

    def __repr__(self):
        return "\n".join(f"{v}: {vizinhos}" for v, vizinhos in self.adj.items())


# criando um grafo não-direcionado
g = Grafo()

g.adicionar_aresta("A", "B")
g.adicionar_aresta("A", "C")
g.adicionar_aresta("B", "D")
g.adicionar_aresta("C", "D")

print("grafo não-direcionado:")
print(g)
print()
print(f"vizinhos de A: {g.vizinhos('A')}")
print(f"vizinhos de D: {g.vizinhos('D')}")

grafo não-direcionado:
A: ['B', 'C']
B: ['A', 'D']
C: ['A', 'D']
D: ['B', 'C']

vizinhos de A: ['B', 'C']
vizinhos de D: ['B', 'C']


## implementação: grafo ponderado

para grafos ponderados, ao invés de guardar só o vizinho, guardamos uma tupla `(vizinho, peso)`.

In [2]:
class GrafoPonderado:
    def __init__(self, direcionado: bool = False):
        self.adj: dict[str, list[tuple[str, int]]] = defaultdict(list)
        self.direcionado = direcionado

    def adicionar_aresta(self, u: str, v: str, peso: int):
        self.adj[u].append((v, peso))

        if not self.direcionado:
            self.adj[v].append((u, peso))

    def vizinhos(self, v: str) -> list[tuple[str, int]]:
        return self.adj[v]

    def __repr__(self):
        linhas: list[str] = []

        for v, vizinhos in self.adj.items():
            conexoes = ", ".join(f"{viz}({peso})" for viz, peso in vizinhos)
            linhas.append(f"{v}: [{conexoes}]")

        return "\n".join(linhas)


# criando um grafo ponderado (distâncias entre cidades)
g = GrafoPonderado()

g.adicionar_aresta("SP", "RJ", 430)
g.adicionar_aresta("SP", "BH", 590)
g.adicionar_aresta("RJ", "BH", 440)
g.adicionar_aresta("BH", "BSB", 740)

print("grafo ponderado (distâncias em km):")
print(g)

grafo ponderado (distâncias em km):
SP: [RJ(430), BH(590)]
RJ: [SP(430), BH(440)]
BH: [SP(590), RJ(440), BSB(740)]
BSB: [BH(740)]


# travessia de grafos

existem duas formas principais de percorrer todos os vértices de um grafo:

1. **BFS (Breadth-First Search):** busca em largura
2. **DFS (Depth-First Search):** busca em profundidade

## BFS — busca em largura

explora todos os vizinhos de um vértice antes de ir para os vizinhos dos vizinhos.  
usa uma **fila (queue)** para manter a ordem de visita.

é útil para:
- encontrar o menor caminho em grafos não-ponderados
- encontrar todos os vértices a uma distância k de um vértice inicial
- verificar se um grafo é bipartido

**complexidade:**
- temporal: `O(V + E)` onde V = vértices, E = arestas
- espacial: `O(V)` para armazenar os vértices visitados

In [3]:
from collections import deque


def bfs(grafo: dict[str, list[str]], inicio: str) -> list[str]:
    visitados: set[str] = set()
    fila: deque[str] = deque([inicio])
    ordem: list[str] = []

    while fila:
        vertice = fila.popleft()

        if vertice not in visitados:
            visitados.add(vertice)
            ordem.append(vertice)

            for vizinho in grafo[vertice]:
                if vizinho not in visitados:
                    fila.append(vizinho)

    return ordem


# grafo de exemplo
#     A --- B
#     |     |
#     C --- D --- E
grafo = {
    "A": ["B", "C"],
    "B": ["A", "D"],
    "C": ["A", "D"],
    "D": ["B", "C", "E"],
    "E": ["D"],
}

print("BFS começando de A:", bfs(grafo, "A"))
print("BFS começando de E:", bfs(grafo, "E"))

BFS começando de A: ['A', 'B', 'C', 'D', 'E']
BFS começando de E: ['E', 'D', 'B', 'C', 'A']


## DFS — busca em profundidade

explora o mais profundo possível antes de voltar e explorar outros caminhos.  
pode ser implementada com **recursão** ou com uma **pilha (stack)**.

é útil para:
- detectar ciclos em grafos
- ordenação topológica
- encontrar componentes conectados
- resolver labirintos

**complexidade:**
- temporal: `O(V + E)`
- espacial: `O(V)` para a pilha de chamadas recursivas

In [4]:
# DFS recursivo
def dfs_recursivo(
    grafo: dict[str, list[str]],
    inicio: str,
    visitados: set[str] | None = None,
) -> list[str]:
    if visitados is None:
        visitados = set()

    visitados.add(inicio)
    ordem = [inicio]

    for vizinho in grafo[inicio]:
        if vizinho not in visitados:
            ordem.extend(dfs_recursivo(grafo, vizinho, visitados))

    return ordem


# DFS iterativo (usando pilha)
def dfs_iterativo(grafo: dict[str, list[str]], inicio: str) -> list[str]:
    visitados: set[str] = set()
    pilha: list[str] = [inicio]
    ordem: list[str] = []

    while pilha:
        vertice = pilha.pop()

        if vertice not in visitados:
            visitados.add(vertice)
            ordem.append(vertice)

            # adiciona vizinhos em ordem reversa para manter a ordem
            for vizinho in reversed(grafo[vertice]):
                if vizinho not in visitados:
                    pilha.append(vizinho)

    return ordem


print("DFS recursivo começando de A:", dfs_recursivo(grafo, "A"))
print("DFS iterativo começando de A:", dfs_iterativo(grafo, "A"))

DFS recursivo começando de A: ['A', 'B', 'D', 'C', 'E']
DFS iterativo começando de A: ['A', 'B', 'D', 'C', 'E']


## BFS vs DFS — quando usar cada um?

| situação                                      | BFS | DFS |
| --------------------------------------------- | --- | --- |
| menor caminho (grafo não-ponderado)           | ✅   | ❌   |
| verificar se existe caminho entre dois nós    | ✅   | ✅   |
| detectar ciclos                               | ✅   | ✅   |
| ordenação topológica                          | ❌   | ✅   |
| encontrar componentes conectados              | ✅   | ✅   |
| resolver labirintos                           | ❌   | ✅   |
| encontrar todos os nós a distância k          | ✅   | ❌   |
| grafo muito profundo (evitar stack overflow)  | ✅   | ❌   |
| grafo muito largo (economizar memória)        | ❌   | ✅   |

# menor caminho

encontrar o menor caminho entre dois vértices é um problema clássico de grafos.

## menor caminho em grafos não-ponderados

para grafos não-ponderados, **BFS** garante o menor caminho, já que explora nível por nível.

In [5]:
def menor_caminho_bfs(
    grafo: dict[str, list[str]],
    inicio: str,
    destino: str,
) -> list[str] | None:
    if inicio == destino:
        return [inicio]

    visitados: set[str] = set([inicio])
    fila: deque[list[str]] = deque([[inicio]])

    while fila:
        caminho = fila.popleft()
        vertice = caminho[-1]

        for vizinho in grafo[vertice]:
            if vizinho == destino:
                return caminho + [vizinho]

            if vizinho not in visitados:
                visitados.add(vizinho)
                fila.append(caminho + [vizinho])

    return None  # não existe caminho


# grafo de exemplo
#     A --- B --- E
#     |     |
#     C --- D
grafo = {
    "A": ["B", "C"],
    "B": ["A", "D", "E"],
    "C": ["A", "D"],
    "D": ["B", "C"],
    "E": ["B"],
}

print(f"menor caminho de A até E: {menor_caminho_bfs(grafo, 'A', 'E')}")
print(f"menor caminho de C até E: {menor_caminho_bfs(grafo, 'C', 'E')}")
print(f"menor caminho de A até D: {menor_caminho_bfs(grafo, 'A', 'D')}")

menor caminho de A até E: ['A', 'B', 'E']
menor caminho de C até E: ['C', 'A', 'B', 'E']
menor caminho de A até D: ['A', 'B', 'D']


## menor caminho em grafos ponderados — dijkstra

para grafos ponderados (com pesos positivos), usamos o **algoritmo de dijkstra**.

a ideia é:
1. começar do vértice inicial com distância 0
2. sempre processar o vértice com menor distância conhecida
3. atualizar as distâncias dos vizinhos se encontrar um caminho menor
4. repetir até processar todos os vértices

usa uma **heap (fila de prioridade)** para sempre pegar o vértice com menor distância.

**complexidade:**
- temporal: `O((V + E) log V)` com heap binária
- espacial: `O(V)`

In [6]:
import heapq


def dijkstra(
    grafo: dict[str, list[tuple[str, int]]],
    inicio: str,
) -> tuple[dict[str, int], dict[str, str | None]]:
    # distâncias iniciais: infinito para todos, exceto o início
    distancias: dict[str, int] = {v: float("inf") for v in grafo}  # type: ignore
    distancias[inicio] = 0

    # predecessor de cada vértice (para reconstruir o caminho)
    anterior: dict[str, str | None] = {v: None for v in grafo}

    # heap de prioridade: (distância, vértice)
    heap: list[tuple[int, str]] = [(0, inicio)]

    while heap:
        dist_atual, vertice = heapq.heappop(heap)

        # se já encontramos um caminho menor, ignorar
        if dist_atual > distancias[vertice]:
            continue

        for vizinho, peso in grafo[vertice]:
            distancia = dist_atual + peso

            # se encontrou um caminho menor, atualizar
            if distancia < distancias[vizinho]:
                distancias[vizinho] = distancia
                anterior[vizinho] = vertice
                heapq.heappush(heap, (distancia, vizinho))

    return distancias, anterior


def reconstruir_caminho(anterior: dict[str, str | None], destino: str) -> list[str]:
    caminho: list[str] = []
    atual: str | None = destino

    while atual is not None:
        caminho.append(atual)
        atual = anterior[atual]

    return list(reversed(caminho))


# grafo ponderado
# SP --430-- RJ
# |          |
# 590       440
# |          |
# BH --740-- BSB
grafo_ponderado: dict[str, list[tuple[str, int]]] = {
    "SP": [("RJ", 430), ("BH", 590)],
    "RJ": [("SP", 430), ("BH", 440)],
    "BH": [("SP", 590), ("RJ", 440), ("BSB", 740)],
    "BSB": [("BH", 740)],
}

distancias, anterior = dijkstra(grafo_ponderado, "SP")

print("distâncias a partir de SP:")
for cidade, dist in distancias.items():
    caminho = reconstruir_caminho(anterior, cidade)
    print(f"  {cidade}: {dist} km — caminho: {' -> '.join(caminho)}")

distâncias a partir de SP:
  SP: 0 km — caminho: SP
  RJ: 430 km — caminho: SP -> RJ
  BH: 590 km — caminho: SP -> BH
  BSB: 1330 km — caminho: SP -> BH -> BSB


# detecção de ciclos

detectar se um grafo contém ciclos é um problema comum.  
a abordagem varia dependendo se o grafo é direcionado ou não.

## ciclo em grafo não-direcionado

um ciclo existe se durante o DFS encontramos um vértice já visitado que não é o pai do vértice atual.

In [7]:
def tem_ciclo_nao_direcionado(grafo: dict[str, list[str]]) -> bool:
    visitados: set[str] = set()

    def dfs(vertice: str, pai: str | None) -> bool:
        visitados.add(vertice)

        for vizinho in grafo[vertice]:
            if vizinho not in visitados:
                if dfs(vizinho, vertice):
                    return True
            elif vizinho != pai:
                # encontrou um vértice visitado que não é o pai = ciclo!
                return True

        return False

    # verificar cada componente (grafo pode ser desconectado)
    for vertice in grafo:
        if vertice not in visitados:
            if dfs(vertice, None):
                return True

    return False


# grafo sem ciclo (árvore)
#     A --- B
#     |
#     C --- D
grafo_sem_ciclo = {
    "A": ["B", "C"],
    "B": ["A"],
    "C": ["A", "D"],
    "D": ["C"],
}

# grafo com ciclo
#     A --- B
#     |     |
#     C --- D
grafo_com_ciclo = {
    "A": ["B", "C"],
    "B": ["A", "D"],
    "C": ["A", "D"],
    "D": ["B", "C"],
}

print(f"grafo sem ciclo tem ciclo? {tem_ciclo_nao_direcionado(grafo_sem_ciclo)}")
print(f"grafo com ciclo tem ciclo? {tem_ciclo_nao_direcionado(grafo_com_ciclo)}")

grafo sem ciclo tem ciclo? False
grafo com ciclo tem ciclo? True


## ciclo em grafo direcionado

para grafos direcionados, usamos três estados:
- **branco:** não visitado
- **cinza:** sendo processado (na pilha de recursão)
- **preto:** completamente processado

se durante o DFS encontramos um vértice **cinza**, existe um ciclo.

In [8]:
from enum import Enum


class Estado(Enum):
    BRANCO = 0  # não visitado
    CINZA = 1  # sendo processado
    PRETO = 2  # completamente processado


def tem_ciclo_direcionado(grafo: dict[str, list[str]]) -> bool:
    estado: dict[str, Estado] = {v: Estado.BRANCO for v in grafo}

    def dfs(vertice: str) -> bool:
        estado[vertice] = Estado.CINZA

        for vizinho in grafo[vertice]:
            if estado[vizinho] == Estado.CINZA:
                # encontrou vértice sendo processado = ciclo!
                return True
            if estado[vizinho] == Estado.BRANCO:
                if dfs(vizinho):
                    return True

        estado[vertice] = Estado.PRETO

        return False

    for vertice in grafo:
        if estado[vertice] == Estado.BRANCO:
            if dfs(vertice):
                return True

    return False


# DAG (sem ciclo)
# A -> B -> C
# |         ^
# +----D----+
dag = {
    "A": ["B", "D"],
    "B": ["C"],
    "C": [],
    "D": ["C"],
}

# grafo direcionado com ciclo
# A -> B -> C
# ^         |
# +---------+
grafo_ciclico = {
    "A": ["B"],
    "B": ["C"],
    "C": ["A"],
}

print(f"DAG tem ciclo? {tem_ciclo_direcionado(dag)}")
print(f"grafo cíclico tem ciclo? {tem_ciclo_direcionado(grafo_ciclico)}")

DAG tem ciclo? False
grafo cíclico tem ciclo? True


# ordenação topológica

ordenação topológica é uma ordenação linear dos vértices de um **DAG** (grafo direcionado acíclico) tal que para toda aresta `u -> v`, `u` vem antes de `v` na ordenação.

é usada para:
- resolver dependências (ex: ordem de compilação, instalação de pacotes)
- scheduling de tarefas
- determinar ordem de cursos pré-requisitos

só funciona em DAGs — se o grafo tiver ciclo, não existe ordenação topológica válida.

In [9]:
def ordenacao_topologica(grafo: dict[str, list[str]]) -> list[str]:
    visitados: set[str] = set()
    pilha: list[str] = []

    def dfs(vertice: str):
        visitados.add(vertice)

        for vizinho in grafo[vertice]:
            if vizinho not in visitados:
                dfs(vizinho)

        # adiciona à pilha após processar todos os vizinhos
        pilha.append(vertice)

    for vertice in grafo:
        if vertice not in visitados:
            dfs(vertice)

    # retorna em ordem reversa
    return list(reversed(pilha))


# dependências de matérias
# calc1 -> calc2 -> calc3
#       -> fisica1 -> fisica2
# algebra -> algebra_linear
dependencias = {
    "calc1": ["calc2", "fisica1"],
    "calc2": ["calc3"],
    "calc3": [],
    "fisica1": ["fisica2"],
    "fisica2": [],
    "algebra": ["algebra_linear"],
    "algebra_linear": [],
}

ordem = ordenacao_topologica(dependencias)

print("ordem de estudo (pré-requisitos primeiro):")
for i, materia in enumerate(ordem, 1):
    print(f"  {i}. {materia}")

ordem de estudo (pré-requisitos primeiro):
  1. algebra
  2. algebra_linear
  3. calc1
  4. fisica1
  5. fisica2
  6. calc2
  7. calc3


# componentes conectados

um **componente conectado** é um subgrafo onde existe um caminho entre qualquer par de vértices.

para grafos não-direcionados, basta fazer BFS/DFS a partir de cada vértice não visitado.

In [10]:
def encontrar_componentes(grafo: dict[str, list[str]]) -> list[list[str]]:
    visitados: set[str] = set()
    componentes: list[list[str]] = []

    def dfs(vertice: str, componente: list[str]):
        visitados.add(vertice)
        componente.append(vertice)

        for vizinho in grafo[vertice]:
            if vizinho not in visitados:
                dfs(vizinho, componente)

    for vertice in grafo:
        if vertice not in visitados:
            componente: list[str] = []
            dfs(vertice, componente)
            componentes.append(componente)

    return componentes


# grafo com 3 componentes desconectados
# componente 1: A - B - C
# componente 2: D - E
# componente 3: F
grafo_desconectado = {
    "A": ["B"],
    "B": ["A", "C"],
    "C": ["B"],
    "D": ["E"],
    "E": ["D"],
    "F": [],
}

componentes = encontrar_componentes(grafo_desconectado)

print(f"número de componentes: {len(componentes)}")
for i, comp in enumerate(componentes, 1):
    print(f"  componente {i}: {comp}")

número de componentes: 3
  componente 1: ['A', 'B', 'C']
  componente 2: ['D', 'E']
  componente 3: ['F']


# complexidade de operações em grafos

| operação                  | lista de adjacência | matriz de adjacência |
| ------------------------- | ------------------- | -------------------- |
| adicionar vértice         | O(1)                | O(V²)                |
| adicionar aresta          | O(1)                | O(1)                 |
| remover vértice           | O(V + E)            | O(V²)                |
| remover aresta            | O(E)                | O(1)                 |
| verificar adjacência      | O(grau)             | O(1)                 |
| encontrar todos vizinhos  | O(grau)             | O(V)                 |
| BFS / DFS                 | O(V + E)            | O(V²)                |
| espaço                    | O(V + E)            | O(V²)                |

## quando usar cada representação

**lista de adjacência** (mais comum):
- grafos esparsos (poucos edges em relação ao máximo possível)
- quando precisa iterar sobre vizinhos frequentemente
- quando o número de vértices pode crescer

**matriz de adjacência:**
- grafos densos (muitos edges)
- quando precisa verificar adjacência frequentemente
- quando o número de vértices é fixo e pequeno

# resumo

grafos são estruturas de dados versáteis para modelar relações entre objetos.

## pontos principais

1. **representações:**
   - lista de adjacência: eficiente para grafos esparsos, `O(V + E)` de espaço
   - matriz de adjacência: eficiente para verificar adjacência, `O(V²)` de espaço

2. **travessias:**
   - BFS: explora em largura, útil para menor caminho em grafos não-ponderados
   - DFS: explora em profundidade, útil para detecção de ciclos e ordenação topológica

3. **algoritmos importantes:**
   - dijkstra: menor caminho em grafos ponderados com pesos positivos
   - ordenação topológica: ordena vértices respeitando dependências (só para DAGs)
   - detecção de ciclos: verifica se existe ciclo no grafo

4. **aplicações:**
   - redes sociais, mapas, dependências de pacotes, scheduling, etc.

## dica prática

na maioria dos casos, use **lista de adjacência** com um `dict[str, list[str]]` ou `defaultdict(list)`.  
é simples, eficiente e cobre a maioria dos casos de uso.